In [1]:
!git clone https://github.com/imatge-upc/speech2signs-2017-nmt.git

Cloning into 'speech2signs-2017-nmt'...
remote: Enumerating objects: 57, done.
remote: Total 57 (delta 0), reused 0 (delta 0), pack-reused 57
Unpacking objects: 100% (57/57), done.


In [4]:
!wget http://www.bu.edu/asllrp/dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx

--2019-09-13 20:29:34--  http://www.bu.edu/asllrp/dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx
Resolving www.bu.edu (www.bu.edu)... 13.32.83.125, 13.32.83.72, 13.32.83.63, ...
Connecting to www.bu.edu (www.bu.edu)|13.32.83.125|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1524218 (1.5M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx’

dai-asllvd-BU_gloss 100%[===================>]   1.45M  1.24MB/s    in 1.2s    

2019-09-13 20:29:36 (1.24 MB/s) - ‘dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx’ saved [1524218/1524218]



In [5]:
!ls

dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx
sample_data
speech2signs-2017-nmt


In [0]:
import pandas as pd

In [0]:
glosses_data = pd.read_excel("dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx")

In [0]:
removed = ['============', False]
glosses = glosses_data["Main New Gloss.1"].dropna().unique()
glosses = [gloss for gloss in glosses if gloss not in removed]

In [107]:
len(glosses)

2746

In [0]:
with open("speech2signs-2017-nmt/ASLG-PC12/ENG-ASL_Train_0.046.asl") as file_obj:
  lines = file_obj.read()
# WOULD LIKE -> WANT
removed_words = [".", ",", "?", "BE", "TO"]
replacement_mapping = {
    "X-I": "IX-1p",
    "X-IT": "IX:i",
    "X-HE": "IX:i",
    "THIS": "IX:i",
    "WILL": "FUTURE",
    "DESC-NOT": "NOT"
}

words = lines.split()
words = [word for word in words if word not in removed_words]
words = [word if word not in replacement_mapping else replacement_mapping[word] for word in words]

In [0]:
word_count = {}
for word in words:
  if word in word_count:
    word_count[word] += 1
  else:
    word_count[word] = 1

In [130]:
glosses_set = set(glosses)
sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

distinct_words = len(sorted_word_count)
total_words = len(words)
word_overlap = [count for word, count in word_count.items() if word in glosses_set]
distinct_word_overlap = len(word_overlap)
total_word_overlap = sum(word_overlap)

print("Vocabulary coverage in ASLG-PC12: {}".format(distinct_word_overlap / distinct_words))
print("Word coverage in ASLG-PC12: {}".format(total_word_overlap / total_words))

Vocabulary coverage in ASLG-PC12: 0.06416166719292705
Word coverage in ASLG-PC12: 0.43827769822180823


In [131]:
[(word, count) for word, count in sorted_word_count[:100] if word not in glosses_set]

[('X-WE', 21806),
 ('EUROPEAN', 5941),
 ('X-YOU', 5820),
 ('X-Y', 4845),
 ('AS', 4280),
 ('DESC-ALSO', 4163),
 ('MR', 3619),
 ('BY', 3552),
 ('DESC-RE', 3512),
 ('NEED', 3507),
 ('WOULD', 3505),
 ('AT', 3384),
 ('SE', 3015),
 ('X-POSS', 2949),
 ('DESC-VERY', 2772),
 ('COMMISSION', 2700),
 ('EU', 2508),
 ('EUROPE', 2434),
 ('DESC-IMPORTANT', 2266),
 ('DESC-HOWEVER', 2210),
 ('UNION', 2175),
 ('DESC-REFORE', 2015),
 ('DEBATE', 1804),
 ('X-MY', 1800),
 ('DESC-NOW', 1789),
 ('NO', 1769),
 ('X-ITS', 1751),
 ('DESC-SO', 1624),
 ('ISSUE', 1589),
 ('DESC-OR', 1526),
 ('COUNCIL', 1514),
 ('THANK', 1510),
 ('DESC-ONLY', 1508),
 ('DESC-NEW', 1480),
 ('CRISIS', 1380),
 ('OR', 1307),
 ('DESC-ECONOMIC', 1243),
 ('DESC-MANY', 1235),
 ('POINT', 1233),
 ('AGREEMENT', 1222),
 ('RIGHTS', 1165),
 ('AREA', 1153),
 ('TODAY', 1143),
 ('DESC-UP', 1102)]

In [0]:
new_glosses = glosses_data["Main New Gloss.1"].unique()

In [123]:
removed = ['============', False]
new_glosses = [gloss for gloss in new_glosses if gloss not in removed]
for gloss in new_glosses:
  if "NOT" in gloss:
    print(gloss)

CANNOT
NOT-CARE
NOT-MIND
ANYWAY/NOT-MATTER
TAKE-NOTES
(1h)NONE/NOTHING
NOTHING
NOTHING-AT-ALL
NOT
NOTICE
NOT-MY-PROBLEM
KNOW-NOTHING
NOT-YET
FEEL+NONE/NOTHING
NOT+SATISFIED
